# Thesis

#### Importing Data From the '19-'20 Season

The first thing we need to do is import all of our necessary libraries. 

In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk


/Users/seandiehl/opt/anaconda3/envs/thesis-env/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


Reading the dataset and importing it into pandas as a DataFrame

In [3]:
PBP20 = pd.read_csv('/Users/seandiehl/Desktop/NBA Play By Play Data/NBA_PBP_2019-20.csv')
pd.set_option('display.max_columns', None)

Here, we are simply inspecting the DataFrame to ensure it looks as expected based on the csv file.

In [4]:
PBP20.head()

,URL,GameType,Location,Date,Time,WinningTeam,Quarter,SecLeft,AwayTeam,AwayPlay,AwayScore,HomeTeam,HomePlay,HomeScore,Shooter,ShotType,ShotOutcome,ShotDist,Assister,Blocker,FoulType,Fouler,Fouled,Rebounder,ReboundType,ViolationPlayer,ViolationType,TimeoutTeam,FreeThrowShooter,FreeThrowOutcome,FreeThrowNum,EnterGame,LeaveGame,TurnoverPlayer,TurnoverType,TurnoverCause,TurnoverCauser,JumpballAwayPlayer,JumpballHomePlayer,JumpballPoss,Unnamed: 40
0,/boxscores/201910220TOR.html,regular,Scotiabank Arena Toronto Canada,October 22 2019,8:00 PM,TOR,1,720,NOP,Jump ball: D. Favors vs. M. Gasol (L. Ball gai...,0,TOR,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D. Favors - favorde01,M. Gasol - gasolma01,L. Ball - balllo01,NaN
1,/boxscores/201910220TOR.html,regular,Scotiabank Arena Toronto Canada,October 22 2019,8:00 PM,TOR,1,708,NOP,L. Ball misses 2-pt jump shot from 11 ft,0,TOR,NaN,0,L. Ball - balllo01,2-pt jump shot,miss,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,/boxscores/201910220TOR.html,regular,Scotiabank Arena Toronto Canada,October 22 2019,8:00 PM,TOR,1,707,NOP,Offensive rebound by D. Favors,0,TOR,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D. Favors - favorde01,offensive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,/boxscores/201910220TOR.html,regular,Scotiabank Arena Toronto Canada,October 22 2019,8:00 PM,TOR,1,707,NOP,D. Favors makes 2-pt layup at rim,2,TOR,NaN,0,D. Favors - favorde01,2-pt layup,make,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,/boxscores/201910220TOR.html,regular,Scotiabank Arena Toronto Canada,October 22 2019,8:00 PM,TOR,1,689,NOP,NaN,2,TOR,O. Anunoby misses 2-pt layup from 3 ft,0,O. Anunoby - anunoog01,2-pt layup,miss,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
PBP20.rename({'Unnamed: 40': 'a'}, axis='columns', inplace=True)
PBP20.drop(['a'], axis=1, inplace=True)

PBP20.head()

,URL,GameType,Location,Date,Time,WinningTeam,Quarter,SecLeft,AwayTeam,AwayPlay,AwayScore,HomeTeam,HomePlay,HomeScore,Shooter,ShotType,ShotOutcome,ShotDist,Assister,Blocker,FoulType,Fouler,Fouled,Rebounder,ReboundType,ViolationPlayer,ViolationType,TimeoutTeam,FreeThrowShooter,FreeThrowOutcome,FreeThrowNum,EnterGame,LeaveGame,TurnoverPlayer,TurnoverType,TurnoverCause,TurnoverCauser,JumpballAwayPlayer,JumpballHomePlayer,JumpballPoss
0,/boxscores/201910220TOR.html,regular,Scotiabank Arena Toronto Canada,October 22 2019,8:00 PM,TOR,1,720,NOP,Jump ball: D. Favors vs. M. Gasol (L. Ball gai...,0,TOR,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D. Favors - favorde01,M. Gasol - gasolma01,L. Ball - balllo01
1,/boxscores/201910220TOR.html,regular,Scotiabank Arena Toronto Canada,October 22 2019,8:00 PM,TOR,1,708,NOP,L. Ball misses 2-pt jump shot from 11 ft,0,TOR,NaN,0,L. Ball - balllo01,2-pt jump shot,miss,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,/boxscores/201910220TOR.html,regular,Scotiabank Arena Toronto Canada,October 22 2019,8:00 PM,TOR,1,707,NOP,Offensive rebound by D. Favors,0,TOR,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D. Favors - favorde01,offensive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,/boxscores/201910220TOR.html,regular,Scotiabank Arena Toronto Canada,October 22 2019,8:00 PM,TOR,1,707,NOP,D. Favors makes 2-pt layup at rim,2,TOR,NaN,0,D. Favors - favorde01,2-pt layup,make,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,/boxscores/201910220TOR.html,regular,Scotiabank Arena Toronto Canada,October 22 2019,8:00 PM,TOR,1,689,NOP,NaN,2,TOR,O. Anunoby misses 2-pt layup from 3 ft,0,O. Anunoby - anunoog01,2-pt layup,miss,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Next, for the relevant columns, stripping the player's Name from the data so that they are left with solely the PlayerID - just makes searching, indexing, etc., a little bit easier down the line.

In [6]:
PBP20['Shooter'] = PBP20['Shooter'].str.split('- ').str[1]
PBP20['Assister'] = PBP20['Assister'].str.split('- ').str[1]
PBP20['Blocker'] = PBP20['Blocker'].str.split('- ').str[1]
PBP20['Fouler'] = PBP20['Fouler'].str.split('- ').str[1]
PBP20['Fouled'] = PBP20['Fouled'].str.split('- ').str[1]
PBP20['Rebounder'] = PBP20['Rebounder'].str.split('- ').str[1]
PBP20['ViolationPlayer'] = PBP20['ViolationPlayer'].str.split('- ').str[1]
PBP20['FreeThrowShooter'] = PBP20['FreeThrowShooter'].str.split('- ').str[1]
PBP20['EnterGame'] = PBP20['EnterGame'].str.split('- ').str[1]
PBP20['LeaveGame'] = PBP20['LeaveGame'].str.split('- ').str[1]
PBP20['TurnoverPlayer'] = PBP20['TurnoverPlayer'].str.split('- ').str[1]
PBP20['TurnoverCauser'] = PBP20['TurnoverCauser'].str.split('- ').str[1]
PBP20['JumpballAwayPlayer'] = PBP20['JumpballAwayPlayer'].str.split('-').str[1]
PBP20['JumpballHomePlayer'] = PBP20['JumpballHomePlayer'].str.split('-').str[1]
PBP20['JumpballPoss'] = PBP20['JumpballPoss'].str.split('-').str[1]

Next, we are going to create a new column, a Unique GameID for each game played throughout the season. <br>

As it stands to reason that each game will have its own unique URL link associated with it, we can simply tell pandas to add 1 to the GameID value each time it comes across a new URL. <br>

Again, this will help with indexing later. 

In [7]:
PBP20['GameID'] = PBP20.groupby('URL').ngroup()


In [8]:
PBP20.head()

,URL,GameType,Location,Date,Time,WinningTeam,Quarter,SecLeft,AwayTeam,AwayPlay,AwayScore,HomeTeam,HomePlay,HomeScore,Shooter,ShotType,ShotOutcome,ShotDist,Assister,Blocker,FoulType,Fouler,Fouled,Rebounder,ReboundType,ViolationPlayer,ViolationType,TimeoutTeam,FreeThrowShooter,FreeThrowOutcome,FreeThrowNum,EnterGame,LeaveGame,TurnoverPlayer,TurnoverType,TurnoverCause,TurnoverCauser,JumpballAwayPlayer,JumpballHomePlayer,JumpballPoss,GameID
0,/boxscores/201910220TOR.html,regular,Scotiabank Arena Toronto Canada,October 22 2019,8:00 PM,TOR,1,720,NOP,Jump ball: D. Favors vs. M. Gasol (L. Ball gai...,0,TOR,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,favorde01,gasolma01,balllo01,1
1,/boxscores/201910220TOR.html,regular,Scotiabank Arena Toronto Canada,October 22 2019,8:00 PM,TOR,1,708,NOP,L. Ball misses 2-pt jump shot from 11 ft,0,TOR,NaN,0,balllo01,2-pt jump shot,miss,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,/boxscores/201910220TOR.html,regular,Scotiabank Arena Toronto Canada,October 22 2019,8:00 PM,TOR,1,707,NOP,Offensive rebound by D. Favors,0,TOR,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,favorde01,offensive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,/boxscores/201910220TOR.html,regular,Scotiabank Arena Toronto Canada,October 22 2019,8:00 PM,TOR,1,707,NOP,D. Favors makes 2-pt layup at rim,2,TOR,NaN,0,favorde01,2-pt layup,make,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,/boxscores/201910220TOR.html,regular,Scotiabank Arena Toronto Canada,October 22 2019,8:00 PM,TOR,1,689,NOP,NaN,2,TOR,O. Anunoby misses 2-pt layup from 3 ft,0,anunoog01,2-pt layup,miss,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


### Seconds Passed in Game Column

Here, we are going to add a column that represents the total seconds passed in the game. We have a column with the value for each quarter, and we have a column with the value for each 

In [9]:
conditions = [
    (PBP20['Quarter'] == 1), (PBP20['Quarter'] == 2), (PBP20['Quarter']==3), (PBP20['Quarter'] ==4), (PBP20['Quarter']==5), (PBP20['Quarter']==6)
]

values = [0, 720, 1440, 2160, 2880, 3180]

PBP20['SecStartQuarter'] = np.select(conditions, values)

PBP20['SecPassed'] = PBP20['SecStartQuarter'] + (720 - PBP20['SecLeft'])

In [11]:
PBP20.head()

,URL,GameType,Location,Date,Time,WinningTeam,Quarter,SecLeft,AwayTeam,AwayPlay,AwayScore,HomeTeam,HomePlay,HomeScore,Shooter,ShotType,ShotOutcome,ShotDist,Assister,Blocker,FoulType,Fouler,Fouled,Rebounder,ReboundType,ViolationPlayer,ViolationType,TimeoutTeam,FreeThrowShooter,FreeThrowOutcome,FreeThrowNum,EnterGame,LeaveGame,TurnoverPlayer,TurnoverType,TurnoverCause,TurnoverCauser,JumpballAwayPlayer,JumpballHomePlayer,JumpballPoss,GameID,SecStartQuarter,SecPassed
0,/boxscores/201910220TOR.html,regular,Scotiabank Arena Toronto Canada,October 22 2019,8:00 PM,TOR,1,720,NOP,Jump ball: D. Favors vs. M. Gasol (L. Ball gai...,0,TOR,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,favorde01,gasolma01,balllo01,1,0,0
1,/boxscores/201910220TOR.html,regular,Scotiabank Arena Toronto Canada,October 22 2019,8:00 PM,TOR,1,708,NOP,L. Ball misses 2-pt jump shot from 11 ft,0,TOR,NaN,0,balllo01,2-pt jump shot,miss,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,12
2,/boxscores/201910220TOR.html,regular,Scotiabank Arena Toronto Canada,October 22 2019,8:00 PM,TOR,1,707,NOP,Offensive rebound by D. Favors,0,TOR,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,favorde01,offensive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,13
3,/boxscores/201910220TOR.html,regular,Scotiabank Arena Toronto Canada,October 22 2019,8:00 PM,TOR,1,707,NOP,D. Favors makes 2-pt layup at rim,2,TOR,NaN,0,favorde01,2-pt layup,make,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,13
4,/boxscores/201910220TOR.html,regular,Scotiabank Arena Toronto Canada,October 22 2019,8:00 PM,TOR,1,689,NOP,NaN,2,TOR,O. Anunoby misses 2-pt layup from 3 ft,0,anunoog01,2-pt layup,miss,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,31


### Lineups - Who's on the court when? 
Next thing we need to do is figure out who is on the court at what times. Because we are working expected average statistics that are adjusted based on the time spent on the court, we need an accurate value for how long, and at what times, each player was on the court during each game.